# Регрессия с Statsmodels. Диагностика

In [ ]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf
from statsmodels.stats import diagnostic # диагностика модели
from statsmodels.stats.stattools import durbin_watson

from statsmodels.iolib.summary2 import summary_col

## Пример: гетероскедастичность для sleep equation

In [ ]:
df = pd.read_csv('sleep75.csv')

На датасете/датафрейме `df` подгоним регрессия `sleep` на `totwrk, age, age^2, male, smsa, marr`.

In [ ]:
# инициализация/спецификация модели через формулу
# создаём объект класса OLS
mod = smf.ols(formula='sleep~totwrk+age+I(age**2)+male+smsa+marr', data=df)

# подгонка специфицированной модели на данных с неробастной ковариационной матрицей для коэффициентов
# создаём объект класса RegressionResults
res = mod.fit()

In [ ]:
# тест на гетероскедастичность
# Вывод: LM-статистика, её P-значение, F-статистика, её P-значение
diagnostic.het_breuschpagan(res.resid, exog_het=mod.exog)

## Пример: серийная корреляция

In [ ]:
df= pd.read_csv('Mishkin.csv')

На датасете/датафрейме `df` подгоним регрессия `pai3` на `tb1, tb3`.

In [ ]:
# инициализация/спецификация модели через формулу
# создаём объект класса OLS
mod = smf.ols(formula='pai3~tb1+tb3', data=df)

# подгонка специфицированной модели на данных с неробастной ковариационной матрицей для коэффициентов
# создаём объект класса RegressionResults
res = mod.fit()

In [ ]:
# Тест на серийную корреляцию второго порядка
# Вывод: LM-статистика, её P-значение, F-статистика, её P-значение
diagnostic.acorr_breusch_godfrey(res=res, nlags=2)

In [ ]:
# Статистика Durbin-Watson
durbin_watson(resids=res.resid)

## Тестирование гипотез с поправкой на гетероскедастичноть и серийную корреляцию
Используем робастную `HAC`-оценку ковариационной матрицы коэффициентов на этапе подгонки модели

In [ ]:
res_hac = mod.fit(cov_type='HAC', cov_kwds={'maxlags': 3})

In [ ]:
summary_col(results=[res, res_hac], model_names=['non-robust', 'robust'], stars=True, info_dict={'F': lambda x: x.fvalue, 'Nobs': lambda x: x.nobs})